# Adatbányászat beadandó

## A modell amivel dolgozunk: 
    dlite-v2-1_5b
### A modell elérhetősége:
    https://huggingface.co/aisquared/dlite-v2-1_5b
### Alap modell:
    https://huggingface.co/openai-community/gpt2-large
### Tanító adathalmaz 01:
    https://huggingface.co/datasets/aisquared/databricks-dolly-15k

### -1 Lépés. Lokális környezet:
    Windows 11 X64
    Intel I7-10750H
    16GB NVIDIA
    GTX1650Ti-4GB

### 0. Lépés: Előfeltételek.
    Rendszergazdai jog ajánlott
	Python <= 3.10.11 (Ennél újabb verzió esetén a Pytorch jelenlegi verziója nem működik)
	Nvdia driver
	Cuda
	Cuda toolkit and devkit

### 1. Lépés: futtatókörnyezet kialakítása
##### Másik meghajtón dolgozunk mert minden elég nagy méretű.
    d:
    cd D:\elte-ik-adatbanyaszat

##### Python virtuális környezetben dolgozunk, hogy ne szemeteljük a gépet, többször próbálkozhassunk.
    python -m venv venv
    venv\Scripts\activate

##### Ajánlott telepítés a cikk szerint, sajnos CPU only, javítsuk.
    pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"

##### Telepítsünk rá 2.2-es torch-ot CUDA 12.1-el.
    pip install torch==2.2.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html

##### Telepítsünk Jupitert, hogy könnyebb dolgunk legyen
    pip install jupyter

##### Indítsuk a Jupitert
    jupyter notebook

## 2. Lépés: a környezet tesztelése

#### Megfelelő a Python verzió?

In [1]:
!python --version

Python 3.10.11


#### Elérhető a CUDA?

In [2]:
!nvidia-smi

Wed Apr 10 21:33:17 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.86                 Driver Version: 551.86         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   37C    P8              4W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### Elérhető a CUDA devkit?

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:30:42_Pacific_Standard_Time_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0


#### Elérhető a Pytorch? És CUDA-val GPU-n?

In [4]:
import torch
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.zeros(1).cuda()

True
cuda:0


tensor([0.], device='cuda:0')

## 3. Lépés: Szerezzük be a modellt lokálisan

In [6]:
!git clone https://huggingface.co/aisquared/dlite-v2-1_5b

Cloning into 'dlite-v2-1_5b'...
Updating files:  66% (8/12)
Updating files:  75% (9/12)
Updating files:  83% (10/12)
Updating files:  91% (11/12)
Updating files: 100% (12/12)
Updating files: 100% (12/12), done.
Filtering content: 100% (2/2), 2.94 GiB | 11.13 MiB/s
Filtering content: 100% (2/2), 2.94 GiB | 11.08 MiB/s, done.


## 4. Lépés: Teszteljük a modellt lokális fájlból GPU-n.

In [1]:
import sys
sys.path.insert(1, './dlite-v2-1_5b/')
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("dlite-v2-1_5b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("dlite-v2-1_5b", device_map="auto", torch_dtype=torch.bfloat16)

#model.to("cuda:0")

generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)

In [6]:
res = generate_text("Who was George Washington?")
print(res)

George Washington was the first United States President.  He served as our first President from 1789 - 1997 and was a military officer in the American Revolutionary War.  He was also a Founding Father and principal author of our Constitution.  During his time in office he created and established many institutions, such as the U.S. government, the U.S. Postal Service, the National Park Service, the National Institutes of Health, and our currency.  Washington is also one of the three presidents who signed the Bill of Rights into law.


#### +1 Teszteljük tovább

In [18]:
res = generate_text("Tell me from Isaac Newton.")
print(res)

Isaac Newton is famous for his three laws of classical mechanics.  According to these three laws, all objects at rest remain at rest, and for every action, there is an equal and opposite reaction.  For example, if I throw a ball, I can expect the ball to hit a ceiling.  If the ball is moving forward at a constant velocity, the acceleration due to gravity is equal to the square of the distance moved, or 3 g.

Newton also discovered two other unique laws that gave him the final two laws of classical mechanics.  First, for every action, there is an equal and opposite reaction. For example, if I throw the ball, the air in front of me pushes the ball away. Second, if a system is in motion, then the force of its motion is equal to its mass multiplied by its velocity.


In [19]:
res = generate_text("Who was Isaac Asimow?")
print(res)

Isaac Asimow (July 23, 1924 – September 17, 2013), known professionally as Isaac Klein, was a singer, songwriter and guitarist who was best known for his years as a member of the rock band R.E.M. He is known for his works with R.E.M. on their album Stuck in the Middle with You (1995), their Movie fame with Their Greatest Hits (2001), and his solo work, including the Grammy Award-winning album Of This World (2005).


In [20]:
res = generate_text("Who wrote the Invincible?")
print(res)

The Invincible is a superhero created by writer Robert Kirkman and artist Tony Moore. It first appeared in the second issue of the Invincible comic book series written by Kirkman and published by Image Comics in 2004.


The Invincible is an American teenager with superpowers. Though he is often at odds with society, the two eventually become best friends and form a superhero team called the Team. The character has been influenced by many popular superheroes, including Spider-Man, Superman, and Wonder Woman.


In [21]:
res = generate_text("Who wrote the Invincible SciFi novel?")
print(res)

Russell Blake wrote the novel Invincible.


In [22]:
res = generate_text("Who was Stanislaw Lem?")
print(res)

Stanislaw Lem is a Polish-American author, best known for his science fiction, including such classics as The Stars, a Star, a Nest and The Silent Planet.


In [23]:
res = generate_text("List Stanislaw Lem writings.")
print(res)

Stanislaw Lem's primary literary genres are the speculative fiction and the memoir.


#### +2 Mennyire alkalmas szövegek összegzésére?

In [2]:
res = generate_text("Summarize this text: The first production helicopter in the Soviet Union was the Mil Mi-1, modelled along the lines of the S-51 and Bristol Sycamore and flown by Mikhail Mil's bureau in September 1948. During the 1950s it became evident, and confirmed by American and French development, that helicopters could be greatly improved with turbine engines. S. P. Isotov developed the GTD-350 engine and Mil used two of these in the far superior Mi-2. The twin shaft-turbine engines used in the Mi-2 develop 40% more power than the Mi-1's piston engines, for barely half the engine weight, with the result that the payload was more than doubled. The Mi-2 fuselage was extensively altered from its predecessor, with the engines mounted overhead. However, the external dimensions remained similar. The Mil-built prototype first flew in the Soviet Union on 22 September 1961, after the initial development the project was transferred to Poland in 1964. The first Świdnik-built example flew on 4 November 1965 (making this the only Soviet-designed helicopter to be built solely outside the Soviet Union). PZL-Świdnik produced a total of 5,497 helicopters, about a third for military users. The factory also developed fiberglass rotor blades, and developed the wide-body Mi-2M seating 10 passengers instead of eight. Most typical role-change kits include four stretchers for air ambulance usage, or aerospraying or cropdusting applications. In Poland, several specialized military variants were also developed in early 1970s for support or training roles, with 23 mm autocannon, machine guns and/or two 57 mm rocket pods, four 9K11 Malyutka anti-tank missiles or Strela-2 AA missiles.")
print(res)

This text provides some background on the Mil Mi-1 helicopter and its upgrade variants. The Mil Mi-1, which began flight testing in September 1948, was a two-seater helicopter powered by a Klimov VK-101 engine with a maximum takeoff weight of 12.2 kg. During its development the Mil bureau gained valuable experience from their experience with the Bristol Sycamore helicopter. During tests in 1949 the helicopter performed well, although a number of manufacturing issues were found and rectified, improving the helicopter's performance during flight.


The Mi-1 first flew in the Soviet Union in September 1961, after the Mil bureau had transferred the project to Poland in 1964. First production models were built in PZL-manufactory in 1965, with total production of about 5,497 helicopters.


In [3]:
res = generate_text("Summarize this text in 1 sentence: The Mikoyan MiG-29 (Russian: Микоян МиГ-29; NATO reporting name: Fulcrum) is a twin-engine fighter aircraft designed in the Soviet Union. Developed by the Mikoyan design bureau as an air superiority fighter during the 1970s, the MiG-29, along with the larger Sukhoi Su-27, was developed to counter new U.S. fighters such as the McDonnell Douglas F-15 Eagle and the General Dynamics F-16 Fighting Falcon.[2] The MiG-29 entered service with the Soviet Air Forces in 1983. While originally oriented towards combat against any enemy aircraft, many MiG-29s have been furnished as multirole fighters capable of performing a number of different operations, and are commonly outfitted to use a range of air-to-surface armaments and precision munitions. The MiG-29 has been manufactured in several major variants, including the multirole Mikoyan MiG-29M and the navalised Mikoyan MiG-29K; the most advanced member of the family to date is the Mikoyan MiG-35. Later models frequently feature improved engines, glass cockpits with HOTAS-compatible flight controls, modern radar and infrared search and track (IRST) sensors, and considerably increased fuel capacity; some aircraft have also been equipped for aerial refueling. Following the dissolution of the Soviet Union, the militaries of multiple ex-Soviet republics have continued to operate the MiG-29, the largest of them being the Russian Aerospace Forces. The Russian Aerospace Forces wanted to upgrade its existing fleet to the modernised MiG-29SMT configuration, but financial difficulties have limited deliveries. The MiG-29 has also been a popular export aircraft; more than 30 nations either operate or have operated the aircraft. As of 2024 Flight Global estimates that 809 MiG-29s, of all types, are in service with air forces, making it the 5th most common active fighter.[3]")
print(res)

The Mikoyan MiG-29 is a Russian fighter aircraft produced in several variants. It first entered service in 1983 and is primarily designed to defeat modern fighters such as the F-15 Eagle and the General Dynamics F-16 Fighting Falcon.


In [4]:
res = generate_text("Summarize this text short: Cassini was one of the most ambitious efforts ever mounted in planetary exploration. A joint endeavor of NASA, ESA (the European Space Agency), and the Italian space agency (ASI), Cassini was a sophisticated robotic spacecraft sent to study Saturn and its complex system of rings and moons in unprecedented detail. Cassini carried a probe called Huygens to the Saturn system. The probe, which was built by ESA, parachuted to the surface of Saturn’s largest moon, Titan, in January 2005—the most distant landing to date in our solar system. Huygens returned spectacular images and other science results during a two-and-a-half-hour descent through Titan’s hazy atmosphere, before coming to rest amid rounded cobbles of ice on a floodplain damp with liquid methane. Cassini completed its initial four-year mission in June 2008 and earned two mission extensions that enabled the team to delve even deeper into Saturn’s mysteries. Key discoveries during its 13 years at Saturn included a global ocean with strong indications of hydrothermal activity within Enceladus and liquid methane seas on Titan. The mission ended on Sept. 15, 2017.")
print(res)

Cassini was a joint project of NASA, ESA (the European Space Agency), and the Italian space agency (ASI). It was an ambitious mission with sophisticated robotic equipment. The mission was 14 years long and ended in September of 2017. The key findings during the mission included a global ocean with strong indications of hydrothermal activity as well as liquid methane seas on Titan.


#### +3 Nem rossz. Vajon ugyanarra a szövegre milyen kéréssel ad jobb kimenetet?

In [2]:
res = generate_text("Summarize this text briefly: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945. The StuG assault guns were cost-effective compared with the heavier German tanks such as the Tiger I and the Panther, although as anti-tank guns they were best used defensively as the lack of a traversable turret and their generally thin armour was a severe disadvantage in the attack role. As the situation for the German military deteriorated further later in the war, more StuGs were built than tanks, particularly due to ease of production. In Italy, the Sturmgeschütz was highly-valued by crews fighting Allied armour, but was dogged by mechanical unreliability; particularly the delicate final drive units. The small box on the track cover, which was normally fixed on the engine deck, contained the track tools.[6] In 1943 and 1944, the Finnish Army received 59 StuG III Ausf. Gs from Germany and used them against the Soviet Union. Thirty of the vehicles were received in 1943 and a further twenty-nine obtained in 1944. The first batch from 1943 destroyed at least eighty-seven enemy tanks for a loss of only eight StuGs (some of which were destroyed by their crews to prevent enemy capture).The later batch from 1944 saw no real action. After the war, the StuGs were the main combat vehicles of the Finnish Army up until the early 1960s when they were phased out. These StuGs gained the nickname Sturmi in the Finnish military, which can be found in some plastic scale-model kits. A StuG III of the Finnish Army in the Heresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added later by the Finnish Army. A StuG III of the Finnish Army in the Heeresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added postwar by the Finnish Army. One hundred StuG III Ausf. Gs were delivered to Romania in the autumn of 1943. They were officially known as TAs (or TAs T3 to avoid confusion with TAs T4 (Jagdpanzer IVs)) in their army's inventory. By February 1945, 13 were still in use with the 2nd Armoured Regiment. None of this initial batch survived the war. Thirty-one TAs were on the Romanian military's inventory in November 1947. Most of them were probably StuG III Ausf. Gs and a small number of Panzer IV/70 (V) (same as TAs T4). These TAs were supplied by the Red Army or were damaged units repaired by the Romanian Army. All German equipment was removed from service in 1950 and finally scrapped four years later due to the army's decision to use only Soviet armour. StuG IIIs were also exported to other nations friendly to Germany, including Bulgaria, Hungary, Italy, and Spain. Hungary fielded its StuG IIIs against Soviet forces as they invaded their country in end-1944 up until early 1945. As with Hungary, Bulgaria received several StuGs from Germany too but almost none saw service against the Soviets, the country having ended the alliance with Germany by switching sides to the Allies before the Soviets invaded.")
print(res)

The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built.


In [3]:
res = generate_text("Summarize this text in 1 sentence: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945. The StuG assault guns were cost-effective compared with the heavier German tanks such as the Tiger I and the Panther, although as anti-tank guns they were best used defensively as the lack of a traversable turret and their generally thin armour was a severe disadvantage in the attack role. As the situation for the German military deteriorated further later in the war, more StuGs were built than tanks, particularly due to ease of production. In Italy, the Sturmgeschütz was highly-valued by crews fighting Allied armour, but was dogged by mechanical unreliability; particularly the delicate final drive units. The small box on the track cover, which was normally fixed on the engine deck, contained the track tools.[6] In 1943 and 1944, the Finnish Army received 59 StuG III Ausf. Gs from Germany and used them against the Soviet Union. Thirty of the vehicles were received in 1943 and a further twenty-nine obtained in 1944. The first batch from 1943 destroyed at least eighty-seven enemy tanks for a loss of only eight StuGs (some of which were destroyed by their crews to prevent enemy capture).The later batch from 1944 saw no real action. After the war, the StuGs were the main combat vehicles of the Finnish Army up until the early 1960s when they were phased out. These StuGs gained the nickname Sturmi in the Finnish military, which can be found in some plastic scale-model kits. A StuG III of the Finnish Army in the Heresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added later by the Finnish Army. A StuG III of the Finnish Army in the Heeresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added postwar by the Finnish Army. One hundred StuG III Ausf. Gs were delivered to Romania in the autumn of 1943. They were officially known as TAs (or TAs T3 to avoid confusion with TAs T4 (Jagdpanzer IVs)) in their army's inventory. By February 1945, 13 were still in use with the 2nd Armoured Regiment. None of this initial batch survived the war. Thirty-one TAs were on the Romanian military's inventory in November 1947. Most of them were probably StuG III Ausf. Gs and a small number of Panzer IV/70 (V) (same as TAs T4). These TAs were supplied by the Red Army or were damaged units repaired by the Romanian Army. All German equipment was removed from service in 1950 and finally scrapped four years later due to the army's decision to use only Soviet armour. StuG IIIs were also exported to other nations friendly to Germany, including Bulgaria, Hungary, Italy, and Spain. Hungary fielded its StuG IIIs against Soviet forces as they invaded their country in end-1944 up until early 1945. As with Hungary, Bulgaria received several StuGs from Germany too but almost none saw service against the Soviets, the country having ended the alliance with Germany by switching sides to the Allies before the Soviets invaded.")
print(res)

The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy.


In [4]:
res = generate_text("Summarize this text short: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945. The StuG assault guns were cost-effective compared with the heavier German tanks such as the Tiger I and the Panther, although as anti-tank guns they were best used defensively as the lack of a traversable turret and their generally thin armour was a severe disadvantage in the attack role. As the situation for the German military deteriorated further later in the war, more StuGs were built than tanks, particularly due to ease of production. In Italy, the Sturmgeschütz was highly-valued by crews fighting Allied armour, but was dogged by mechanical unreliability; particularly the delicate final drive units. The small box on the track cover, which was normally fixed on the engine deck, contained the track tools.[6] In 1943 and 1944, the Finnish Army received 59 StuG III Ausf. Gs from Germany and used them against the Soviet Union. Thirty of the vehicles were received in 1943 and a further twenty-nine obtained in 1944. The first batch from 1943 destroyed at least eighty-seven enemy tanks for a loss of only eight StuGs (some of which were destroyed by their crews to prevent enemy capture).The later batch from 1944 saw no real action. After the war, the StuGs were the main combat vehicles of the Finnish Army up until the early 1960s when they were phased out. These StuGs gained the nickname Sturmi in the Finnish military, which can be found in some plastic scale-model kits. A StuG III of the Finnish Army in the Heresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added later by the Finnish Army. A StuG III of the Finnish Army in the Heeresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added postwar by the Finnish Army. One hundred StuG III Ausf. Gs were delivered to Romania in the autumn of 1943. They were officially known as TAs (or TAs T3 to avoid confusion with TAs T4 (Jagdpanzer IVs)) in their army's inventory. By February 1945, 13 were still in use with the 2nd Armoured Regiment. None of this initial batch survived the war. Thirty-one TAs were on the Romanian military's inventory in November 1947. Most of them were probably StuG III Ausf. Gs and a small number of Panzer IV/70 (V) (same as TAs T4). These TAs were supplied by the Red Army or were damaged units repaired by the Romanian Army. All German equipment was removed from service in 1950 and finally scrapped four years later due to the army's decision to use only Soviet armour. StuG IIIs were also exported to other nations friendly to Germany, including Bulgaria, Hungary, Italy, and Spain. Hungary fielded its StuG IIIs against Soviet forces as they invaded their country in end-1944 up until early 1945. As with Hungary, Bulgaria received several StuGs from Germany too but almost none saw service against the Soviets, the country having ended the alliance with Germany by switching sides to the Allies before the Soviets invaded.")
print(res)

Sturmgeschütz III - series vehicles were extremely successful and served on all fronts from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy.


In [5]:
res = generate_text("What is this text about: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945. The StuG assault guns were cost-effective compared with the heavier German tanks such as the Tiger I and the Panther, although as anti-tank guns they were best used defensively as the lack of a traversable turret and their generally thin armour was a severe disadvantage in the attack role. As the situation for the German military deteriorated further later in the war, more StuGs were built than tanks, particularly due to ease of production. In Italy, the Sturmgeschütz was highly-valued by crews fighting Allied armour, but was dogged by mechanical unreliability; particularly the delicate final drive units. The small box on the track cover, which was normally fixed on the engine deck, contained the track tools.[6] In 1943 and 1944, the Finnish Army received 59 StuG III Ausf. Gs from Germany and used them against the Soviet Union. Thirty of the vehicles were received in 1943 and a further twenty-nine obtained in 1944. The first batch from 1943 destroyed at least eighty-seven enemy tanks for a loss of only eight StuGs (some of which were destroyed by their crews to prevent enemy capture).The later batch from 1944 saw no real action. After the war, the StuGs were the main combat vehicles of the Finnish Army up until the early 1960s when they were phased out. These StuGs gained the nickname Sturmi in the Finnish military, which can be found in some plastic scale-model kits. A StuG III of the Finnish Army in the Heresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added later by the Finnish Army. A StuG III of the Finnish Army in the Heeresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added postwar by the Finnish Army. One hundred StuG III Ausf. Gs were delivered to Romania in the autumn of 1943. They were officially known as TAs (or TAs T3 to avoid confusion with TAs T4 (Jagdpanzer IVs)) in their army's inventory. By February 1945, 13 were still in use with the 2nd Armoured Regiment. None of this initial batch survived the war. Thirty-one TAs were on the Romanian military's inventory in November 1947. Most of them were probably StuG III Ausf. Gs and a small number of Panzer IV/70 (V) (same as TAs T4). These TAs were supplied by the Red Army or were damaged units repaired by the Romanian Army. All German equipment was removed from service in 1950 and finally scrapped four years later due to the army's decision to use only Soviet armour. StuG IIIs were also exported to other nations friendly to Germany, including Bulgaria, Hungary, Italy, and Spain. Hungary fielded its StuG IIIs against Soviet forces as they invaded their country in end-1944 up until early 1945. As with Hungary, Bulgaria received several StuGs from Germany too but almost none saw service against the Soviets, the country having ended the alliance with Germany by switching sides to the Allies before the Soviets invaded.")
print(res)

This text is about: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built.


#### Melyik promt a legjobb eddig?
Eddig ez a legjobb: Summarize this text in 1 sentence: 

## 5. Lépés: Próbáljuk meg tanítani a saját adathalmazunkkal?

Megnézendő: https://huggingface.co/docs/transformers/en/training
Tanítsuk egy kérdés adatbázissal? Van egy.
CSV-ben társítva.

Esetleg használható dataset: https://huggingface.co/datasets/mandarjoshi/trivia_qa

Ötletek alkalmazáshoz:

Chatbot: Készíthetsz egy chatbotot, amely képes valós idejű kommunikációra és válaszadásra felhasználókkal. Ez lehet egy általános célú chatbot, vagy specializáltabb, például egy tech support vagy tanácsadó chatbot.

Tartalom generálás: Használhatod a GPT-2-t új tartalmak generálására, például cikkek, novellák, versek stb. létrehozására. Ez különösen hasznos lehet, ha olyan platformot üzemeltetsz, amely folyamatosan friss tartalmakra van szükség.

Automatikus cím generálás: Fejleszthetsz egy rendszert, amely automatikusan generál címeket cikkekhez vagy blogbejegyzésekhez a megadott témában.

Segéd a kreatív íráshoz: Készíthetsz egy eszközt, amely segít a kreatív írásban, például ötleteket ad szókincsre, témákra vagy cselekményekre.

Kérdés-válasz rendszer: Implementálhatsz egy rendszert, amely képes válaszolni felhasználók által feltett kérdésekre. Ez hasznos lehet például FAQ oldalakon vagy oktató rendszerekben.

Idézet generálás: Készíthetsz egy alkalmazást, amely idézeteket generál különböző témákban vagy hírességektől.

Nyelvi modellek finomhangolása: Finomhangolhatod a GPT-2-t saját adatokon vagy speciális feladatokra, például jogi dokumentumok, orvosi szövegek vagy pénzügyi jelentések kezelésére.

Párbeszédgenerálás játékokban: Integrálhatod a GPT-2-t párbeszédgenerálásra játékokba, interaktív történetekbe vagy szerepjátékokba.

Reklám- vagy termékleírás-generálás: Automatizálhatod a reklám vagy termékleírások generálását az egyes termékek vagy szolgáltatások alapján.

Nyelvi támogatás: Készíthetsz egy alkalmazást, amely nyelvi segítséget nyújt az embereknek, például nyelvtanuláshoz vagy fordításhoz.

Ötlet: Versíró segéd? Kulcsszavakra részleteket generál?
Szöveg rövidítő, összegző.
Cím generátor.

Ezekhez dataset?

### Szöveg összegzés tesztelése sikeres volt. Erre jó lenne.

#### Chrome extension-t kelene fejleszteni:

https://developer.chrome.com/docs/extensions/get-started/tutorial/hello-world

https://stackoverflow.com/questions/4376167/text-selection-and-display-in-context-menu-chrome-extension

#### Tudjon többet is?
* Foglald össze OK Tesztelve
* Adj kulcsszavakat
* Adj címet neki
  Teszteljük ezeket


In [6]:
res = generate_text("Give this text a title: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945. The StuG assault guns were cost-effective compared with the heavier German tanks such as the Tiger I and the Panther, although as anti-tank guns they were best used defensively as the lack of a traversable turret and their generally thin armour was a severe disadvantage in the attack role. As the situation for the German military deteriorated further later in the war, more StuGs were built than tanks, particularly due to ease of production. In Italy, the Sturmgeschütz was highly-valued by crews fighting Allied armour, but was dogged by mechanical unreliability; particularly the delicate final drive units. The small box on the track cover, which was normally fixed on the engine deck, contained the track tools.[6] In 1943 and 1944, the Finnish Army received 59 StuG III Ausf. Gs from Germany and used them against the Soviet Union. Thirty of the vehicles were received in 1943 and a further twenty-nine obtained in 1944. The first batch from 1943 destroyed at least eighty-seven enemy tanks for a loss of only eight StuGs (some of which were destroyed by their crews to prevent enemy capture).The later batch from 1944 saw no real action. After the war, the StuGs were the main combat vehicles of the Finnish Army up until the early 1960s when they were phased out. These StuGs gained the nickname Sturmi in the Finnish military, which can be found in some plastic scale-model kits. A StuG III of the Finnish Army in the Heresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added later by the Finnish Army. A StuG III of the Finnish Army in the Heeresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added postwar by the Finnish Army. One hundred StuG III Ausf. Gs were delivered to Romania in the autumn of 1943. They were officially known as TAs (or TAs T3 to avoid confusion with TAs T4 (Jagdpanzer IVs)) in their army's inventory. By February 1945, 13 were still in use with the 2nd Armoured Regiment. None of this initial batch survived the war. Thirty-one TAs were on the Romanian military's inventory in November 1947. Most of them were probably StuG III Ausf. Gs and a small number of Panzer IV/70 (V) (same as TAs T4). These TAs were supplied by the Red Army or were damaged units repaired by the Romanian Army. All German equipment was removed from service in 1950 and finally scrapped four years later due to the army's decision to use only Soviet armour. StuG IIIs were also exported to other nations friendly to Germany, including Bulgaria, Hungary, Italy, and Spain. Hungary fielded its StuG IIIs against Soviet forces as they invaded their country in end-1944 up until early 1945. As with Hungary, Bulgaria received several StuGs from Germany too but almost none saw service against the Soviets, the country having ended the alliance with Germany by switching sides to the Allies before the Soviets invaded.")
print(res)

The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built.


In [7]:
res = generate_text("Give keywords for this text: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945. The StuG assault guns were cost-effective compared with the heavier German tanks such as the Tiger I and the Panther, although as anti-tank guns they were best used defensively as the lack of a traversable turret and their generally thin armour was a severe disadvantage in the attack role. As the situation for the German military deteriorated further later in the war, more StuGs were built than tanks, particularly due to ease of production. In Italy, the Sturmgeschütz was highly-valued by crews fighting Allied armour, but was dogged by mechanical unreliability; particularly the delicate final drive units. The small box on the track cover, which was normally fixed on the engine deck, contained the track tools.[6] In 1943 and 1944, the Finnish Army received 59 StuG III Ausf. Gs from Germany and used them against the Soviet Union. Thirty of the vehicles were received in 1943 and a further twenty-nine obtained in 1944. The first batch from 1943 destroyed at least eighty-seven enemy tanks for a loss of only eight StuGs (some of which were destroyed by their crews to prevent enemy capture).The later batch from 1944 saw no real action. After the war, the StuGs were the main combat vehicles of the Finnish Army up until the early 1960s when they were phased out. These StuGs gained the nickname Sturmi in the Finnish military, which can be found in some plastic scale-model kits. A StuG III of the Finnish Army in the Heresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added later by the Finnish Army. A StuG III of the Finnish Army in the Heeresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added postwar by the Finnish Army. One hundred StuG III Ausf. Gs were delivered to Romania in the autumn of 1943. They were officially known as TAs (or TAs T3 to avoid confusion with TAs T4 (Jagdpanzer IVs)) in their army's inventory. By February 1945, 13 were still in use with the 2nd Armoured Regiment. None of this initial batch survived the war. Thirty-one TAs were on the Romanian military's inventory in November 1947. Most of them were probably StuG III Ausf. Gs and a small number of Panzer IV/70 (V) (same as TAs T4). These TAs were supplied by the Red Army or were damaged units repaired by the Romanian Army. All German equipment was removed from service in 1950 and finally scrapped four years later due to the army's decision to use only Soviet armour. StuG IIIs were also exported to other nations friendly to Germany, including Bulgaria, Hungary, Italy, and Spain. Hungary fielded its StuG IIIs against Soviet forces as they invaded their country in end-1944 up until early 1945. As with Hungary, Bulgaria received several StuGs from Germany too but almost none saw service against the Soviets, the country having ended the alliance with Germany by switching sides to the Allies before the Soviets invaded.")
print(res)

Sturmgeschütz, Sturmgeschütz, The Sturmgeschütz, III series, vehicles, Allied forces, Soviet Union


In [8]:
res = generate_text("Give this text a 2 word tittle: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945. The StuG assault guns were cost-effective compared with the heavier German tanks such as the Tiger I and the Panther, although as anti-tank guns they were best used defensively as the lack of a traversable turret and their generally thin armour was a severe disadvantage in the attack role. As the situation for the German military deteriorated further later in the war, more StuGs were built than tanks, particularly due to ease of production. In Italy, the Sturmgeschütz was highly-valued by crews fighting Allied armour, but was dogged by mechanical unreliability; particularly the delicate final drive units. The small box on the track cover, which was normally fixed on the engine deck, contained the track tools.[6] In 1943 and 1944, the Finnish Army received 59 StuG III Ausf. Gs from Germany and used them against the Soviet Union. Thirty of the vehicles were received in 1943 and a further twenty-nine obtained in 1944. The first batch from 1943 destroyed at least eighty-seven enemy tanks for a loss of only eight StuGs (some of which were destroyed by their crews to prevent enemy capture).The later batch from 1944 saw no real action. After the war, the StuGs were the main combat vehicles of the Finnish Army up until the early 1960s when they were phased out. These StuGs gained the nickname Sturmi in the Finnish military, which can be found in some plastic scale-model kits. A StuG III of the Finnish Army in the Heresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added later by the Finnish Army. A StuG III of the Finnish Army in the Heeresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added postwar by the Finnish Army. One hundred StuG III Ausf. Gs were delivered to Romania in the autumn of 1943. They were officially known as TAs (or TAs T3 to avoid confusion with TAs T4 (Jagdpanzer IVs)) in their army's inventory. By February 1945, 13 were still in use with the 2nd Armoured Regiment. None of this initial batch survived the war. Thirty-one TAs were on the Romanian military's inventory in November 1947. Most of them were probably StuG III Ausf. Gs and a small number of Panzer IV/70 (V) (same as TAs T4). These TAs were supplied by the Red Army or were damaged units repaired by the Romanian Army. All German equipment was removed from service in 1950 and finally scrapped four years later due to the army's decision to use only Soviet armour. StuG IIIs were also exported to other nations friendly to Germany, including Bulgaria, Hungary, Italy, and Spain. Hungary fielded its StuG IIIs against Soviet forces as they invaded their country in end-1944 up until early 1945. As with Hungary, Bulgaria received several StuGs from Germany too but almost none saw service against the Soviets, the country having ended the alliance with Germany by switching sides to the Allies before the Soviets invaded.")
print(res)

The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945.


### Mi kell? Gyűjtsük össze.

Sajnos 1024-es token limit van, kell egy token counter. Nézzük csak.


In [16]:
tokens = tokenizer("Give keywords for this text: The Sturmgeschütz III-series of vehicles proved very successful and served on all fronts, from Russia to North Africa and Western Europe to Italy, as assault guns and tank destroyers. Because of their low silhouette, StuG IIIs were easy to camouflage and hide, and were difficult targets to destroy. By the end of the war 11,300 StuG IIIs and StuH 42s had been built. but due to heavy losses, there were only 1,053 StuG IIIs and 277 StuH 42s remaining in German service by 10 April 1945. The StuG assault guns were cost-effective compared with the heavier German tanks such as the Tiger I and the Panther, although as anti-tank guns they were best used defensively as the lack of a traversable turret and their generally thin armour was a severe disadvantage in the attack role. As the situation for the German military deteriorated further later in the war, more StuGs were built than tanks, particularly due to ease of production. In Italy, the Sturmgeschütz was highly-valued by crews fighting Allied armour, but was dogged by mechanical unreliability; particularly the delicate final drive units. The small box on the track cover, which was normally fixed on the engine deck, contained the track tools.[6] In 1943 and 1944, the Finnish Army received 59 StuG III Ausf. Gs from Germany and used them against the Soviet Union. Thirty of the vehicles were received in 1943 and a further twenty-nine obtained in 1944. The first batch from 1943 destroyed at least eighty-seven enemy tanks for a loss of only eight StuGs (some of which were destroyed by their crews to prevent enemy capture).The later batch from 1944 saw no real action. After the war, the StuGs were the main combat vehicles of the Finnish Army up until the early 1960s when they were phased out. These StuGs gained the nickname Sturmi in the Finnish military, which can be found in some plastic scale-model kits. A StuG III of the Finnish Army in the Heresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added later by the Finnish Army. A StuG III of the Finnish Army in the Heeresgeschichtliches Museum in Vienna, Austria. This model has the concrete armor added postwar by the Finnish Army. One hundred StuG III Ausf. Gs were delivered to Romania in the autumn of 1943. They were officially known as TAs (or TAs T3 to avoid confusion with TAs T4 (Jagdpanzer IVs)) in their army's inventory. By February 1945, 13 were still in use with the 2nd Armoured Regiment. None of this initial batch survived the war. Thirty-one TAs were on the Romanian military's inventory in November 1947. Most of them were probably StuG III Ausf. Gs and a small number of Panzer IV/70 (V) (same as TAs T4). These TAs were supplied by the Red Army or were damaged units repaired by the Romanian Army. All German equipment was removed from service in 1950 and finally scrapped four years later due to the army's decision to use only Soviet armour. StuG IIIs were also exported to other nations friendly to Germany, including Bulgaria, Hungary, Italy, and Spain. Hungary fielded its StuG IIIs against Soviet forces as they invaded their country in end-1944 up until early 1945. As with Hungary, Bulgaria received several StuGs from Germany too but almost none saw service against the Soviets, the country having ended the alliance with Germany by switching sides to the Allies before the Soviets invaded.")
print(tokens)

{'input_ids': [23318, 26286, 329, 428, 2420, 25, 383, 26783, 76, 3212, 354, 9116, 22877, 6711, 12, 25076, 286, 5672, 8302, 845, 4388, 290, 4983, 319, 477, 29324, 11, 422, 3284, 284, 2258, 5478, 290, 4885, 2031, 284, 8031, 11, 355, 4641, 6541, 290, 6873, 4117, 364, 13, 4362, 286, 511, 1877, 41834, 11, 520, 84, 38, 2873, 3792, 547, 2562, 284, 37353, 290, 7808, 11, 290, 547, 2408, 6670, 284, 4117, 13, 2750, 262, 886, 286, 262, 1175, 1367, 11, 6200, 520, 84, 38, 2873, 3792, 290, 520, 84, 39, 5433, 82, 550, 587, 3170, 13, 475, 2233, 284, 4334, 9089, 11, 612, 547, 691, 352, 11, 2713, 18, 520, 84, 38, 2873, 3792, 290, 38703, 520, 84, 39, 5433, 82, 5637, 287, 2679, 2139, 416, 838, 3035, 15761, 13, 383, 520, 84, 38, 4641, 6541, 547, 1575, 12, 16803, 3688, 351, 262, 20140, 2679, 11657, 884, 355, 262, 17030, 314, 290, 262, 30364, 11, 3584, 355, 3098, 12, 28451, 6541, 484, 547, 1266, 973, 33352, 355, 262, 3092, 286, 257, 33038, 540, 28101, 290, 511, 4143, 7888, 18588, 373, 257, 6049, 21407, 287, 2

In [19]:
tokens = tokenizer("Lorem ipsum dolor sit amet, consectetur adipiscing elit. In a ipsum eget erat pharetra facilisis ac sit amet massa. Nulla venenatis nisl diam. Cras feugiat fermentum mauris, vel semper enim blandit aliquam. Quisque laoreet, nisl in aliquet hendrerit, justo mi volutpat nisi, sed molestie ligula turpis sed est. Quisque vitae tortor laoreet mi pulvinar luctus quis et eros. Suspendisse efficitur, nisl in auctor dignissim, erat velit interdum nulla, nec volutpat lacus mauris feugiat mauris. Pellentesque et est lobortis, pulvinar libero vel, tempor sem. Integer tempus sagittis erat tempus ultrices. Ut luctus, ante vel feugiat mattis, lectus augue gravida ex, id ultricies est tortor sit amet justo. Vivamus accumsan libero lorem, eu rhoncus purus blandit in. Sed in nibh tristique, iaculis risus in, dignissim lorem. In hac habitasse platea dictumst. Praesent tristique sit amet ante eu ultrices. Sed commodo luctus rhoncus. Phasellus at tellus at dolor consequat bibendum. Pellentesque ut mauris tempus odio lobortis pharetra eget vel ligula. Donec commodo maximus libero quis finibus. Pellentesque vehicula gravida commodo. Vivamus elementum sit amet nisl non vulputate. Aliquam nec velit pretium, congue libero a, imperdiet eros. Praesent egestas sem vitae justo maximus, vitae convallis metus bibendum. Pellentesque nec luctus libero, vitae auctor lorem. Mauris eros massa, placerat a fringilla tincidunt, eleifend pretium ipsum. Nulla euismod elit non nulla pellentesque, vel dignissim mi vestibulum. Curabitur tincidunt libero vel suscipit pellentesque. Duis lacinia at leo eu dictum. Curabitur dapibus tristique orci quis pharetra. Praesent vel venenatis erat. Donec tempus, libero a tincidunt elementum, odio nulla scelerisque quam, sed molestie odio arcu a elit. Nunc nec metus turpis. Sed ornare, mauris id scelerisque placerat, nibh nisl blandit neque, sit amet imperdiet lectus justo mattis ipsum. Curabitur sit amet mi ex. Cras quis lobortis libero. Mauris eget nisl vestibulum nisl gravida egestas. Vivamus lobortis dictum mi at sollicitudin. Donec facilisis, ante at condimentum aliquet, nisl ex rhoncus est, id dignissim sem eros nec nisi. Suspendisse pulvinar, risus quis malesuada varius, enim libero vehicula justo, at consequat ipsum mauris nec odio. Integer at turpis rhoncus, fringilla augue sed, vestibulum mauris. Vivamus aliquet augue nisl, non accumsan urna cursus eget. Praesent porttitor porta arcu at fermentum. Mauris nec lobortis metus, eu faucibus nulla. Integer nisl mi, congue quis condimentum vitae, consequat et ipsum. Nulla sit amet fringilla mi, ut placerat quam. Aliquam a erat purus. Cras ac neque ornare, semper ligula id, posuere sem. Etiam feugiat tortor sit amet feugiat semper. Cras elementum tempus nunc, quis molestie diam aliquam id. Duis vitae venenatis mi. Ut vitae auctor lectus, quis hendrerit sem. Suspendisse at egestas dui. Ut molestie ante in luctus elementum. Aenean efficitur at lacus id gravida. Vivamus ultricies, dui ut posuere feugiat, sem metus vulputate ligula, id rutrum libero metus nec purus. Sed bibendum libero lacus, in venenatis elit pellentesque quis. Nam vitae eleifend est, sed viverra magna. Ut sit amet orci dictum, porttitor ante sed, ultricies sem. Curabitur at magna ac quam cursus tincidunt vitae eget risus. Sed at turpis erat. Vivamus at aliquam libero, sed rutrum enim. Suspendisse lorem mauris, molestie ac tempus quis, mollis sed est. Sed scelerisque urna nec urna condimentum, elementum mattis neque dignissim. Mauris pretium tincidunt est, ac viverra sem ultricies eu. Quisque vitae lacus at arcu aliquet imperdiet. Morbi nec elit nulla. Donec non pellentesque turpis, eget sodales arcu. Fusce sit amet dolor vel velit accumsan facilisis quis at massa. Cras aliquam tortor ac nulla mollis molestie. Cras ac tortor quis nisi fringilla ultrices. Nunc convallis ultrices arcu non feugiat. Nam non arcu odio. Duis viverra efficitur eros, at malesuada quam pretium ut. Vivamus facilisis vitae tortor nec volutpat. Phasellus efficitur mollis mauris. Quisque id augue neque. Quisque maximus odio non nunc tristique luctus. Phasellus justo quam, iaculis eget eros sit amet, posuere luctus diam. Proin dolor neque, ultrices sit amet risus at, bibendum maximus lectus. Donec urna turpis, malesuada id nibh vitae, pretium suscipit quam. Cras non consequat massa. Mauris pulvinar et metus ut viverra. Mauris ut laoreet metus. Nunc magna erat, fringilla ac augue ut, iaculis cursus magna. Nullam eu velit vel lorem tincidunt accumsan vitae eu nisl. Ut consequat sodales ipsum, in tempus tortor luctus varius. Sed aliquam tellus vitae lorem pellentesque malesuada. Proin scelerisque nibh sit amet nunc lobortis tincidunt. Etiam sapien lacus, placerat nec ex sed, ultricies hendrerit urna. Proin aliquam aliquet mattis. Phasellus luctus mollis ex, id pharetra ante feugiat et. Suspendisse potenti. Nunc interdum malesuada est ut fringilla. Pellentesque sed sapien eget risus scelerisque congue. Cras vulputate mauris eget rhoncus porttitor. Mauris aliquam ornare pellentesque. Curabitur quis nibh ligula. Donec ornare ipsum urna, eu tempus orci feugiat non. In vulputate nisl id sem mollis, in accumsan nulla dapibus. Fusce accumsan, purus sit amet tristique volutpat, neque turpis ornare enim, vel elementum est nibh in odio. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Praesent ac urna massa. Praesent at eros ornare, vehicula elit quis, vestibulum ex. Mauris accumsan eu enim at commodo. Integer blandit id justo sit amet mollis. Donec vel orci erat. Mauris sed leo nec dolor fringilla fermentum. Integer odio nibh, convallis ac tincidunt eu, posuere vitae justo. Quisque diam eros, ullamcorper at tortor non, tristique blandit felis. Fusce non nunc ac risus vestibulum eleifend. Nunc at tortor eget neque pulvinar cursus sed id dolor. In lobortis dui tincidunt justo vehicula finibus. Nam laoreet nulla nec sagittis posuere. Aenean risus arcu, pretium in erat non, rhoncus volutpat tellus. Donec varius et neque id facilisis. Morbi commodo turpis non mi ultrices, id fermentum lacus scelerisque. Ut faucibus pharetra ornare. Proin sit amet eros nec quam suscipit semper. Quisque nec lectus sem. Curabitur sit amet augue dui. Nulla in ex aliquet, pellentesque lectus consequat, volutpat tellus. Nullam fermentum tellus elit. Sed ullamcorper ligula turpis, at luctus lorem faucibus id. Sed euismod lorem non malesuada congue. In hac habitasse platea dictumst. In ultricies, felis in consectetur dignissim, purus dolor blandit orci, nec sodales ligula turpis nec tortor. Nulla sit amet libero id arcu luctus egestas eget id sem. In hac habitasse platea dictumst. Cras sodales, ipsum non tincidunt sodales, elit ipsum ultricies nunc, at efficitur lorem metus sed odio. Nunc sodales, mauris in elementum.")
print(len(tokens.attention_mask))

Token indices sequence length is longer than the specified maximum sequence length for this model (2396 > 1024). Running this sequence through the model will result in indexing errors


2396


#### Ez jó lesz ellenőrzésre

Kell egy webszerver ami GET kérésre válaszolni tud. Nézzük csak.

In [22]:
!pip install flask

     ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
     ----------------------------------- --- 92.2/101.7 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 101.7/101.7 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/226.8 kB ? eta -:--:--
     -------------------- ----------------- 122.9/226.8 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 226.8/226.8 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
     ---------------------------------------- 97.9/97.9 kB 5.8 MB/s eta 0:00:00


In [25]:
from flask import Flask, jsonify
import subprocess

app = Flask(__name__)

@app.route('/date', methods=['GET'])
def get_date():
    return jsonify({'date': 'iTS WORKING'})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Apr/2024 22:54:59] "GET /date HTTP/1.1" 200 -


In [3]:
from flask import Flask, jsonify
import subprocess

app = Flask(__name__)

@app.route('/req', methods=['POST'])
def get_req():
    data = request.get_json()
    text = data.get('text', '')
    #ret = {'resp': generate_text("Summarize this text in 1 sentence:" + text)}
    ret = {'resp2': generate_text("Give keywords for this text: " + text)}
    return jsonify(ret)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on /req [POST]
Traceback (most recent call last):
  File "D:\elte-ik-adatbanyaszat\venv\lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\elte-ik-adatbanyaszat\venv\lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\elte-ik-adatbanyaszat\venv\lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\elte-ik-adatbanyaszat\venv\lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\AD\AppData\Local\Temp\ipykernel_16184\3169928668.py", line 8, in get_req


## OK Az API működik, persze még kell bele token max és szöveg tisztítás, spec karakterek kivétele de már megy.

In [1]:
!pip install wrapt_timeout_decorator

     ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
     -------------------------------------- 116.3/116.3 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     -------------------------------------- 134.8/134.8 kB 4.0 MB/s eta 0:00:00


Jöhet a chrome extension. Ezzel a ChatGPT bemenettel egész jól legenerálja
Hi!
How can i create a chrome extension which enables you to select text on any given website and displays a right click option to send the selection to a REST POST api and show the rsoult.



In [1]:
!pip install accelerate

In [1]:
pip install flask-cors

Note: you may need to restart the kernel to use updated packages.
